# 第三章 提示优化工具

<div class="toc">
 <ul class="toc-item">
     <li><span><a href="#一、引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
     <li>
         <span><a href="#二、优化 Prompt 的提示" data-toc-modified-id="二、优化 Prompt 的提示">二、优化 Prompt 的提示</a></span>
         <ul class="toc-item">
             <li><span><a href="##2.1 示例 1" data-toc-modified-id="2.1 示例 1">2.1 示例 1</a></span></li>
             <li><span><a href="##2.2 示例 2" data-toc-modified-id="2.2 示例 2">2.2 示例 2</a></span></li>
             <li><span><a href="##2.3 示例 3" data-toc-modified-id="2.3 示例 3">2.3 示例 3</a></span></li>
             </ul>
         </li>
     <li><span><a href="#三、总结与讨论" data-toc-modified-id="三、总结与讨论">三、总结与讨论</a></span></li>
     </ul>
</div>

## 一、引言
***

对于大语言模型（Large Language Models, LLMs），**我们通常需要遵循一些原则和策略来创建能产生高质量回答的有效提示：**

1. 明确和具体：你应该尽可能明确和具体地描述你希望 LLM 执行的任务。如果你期望得到特定类型的回答，应在提示中明确指出。同时，如果存在任何具体的限制或要求，也应明确提出。

2. 开放式与封闭式：你可以根据需求选择开放式问题（允许多种回答）或封闭式问题（限制回答范围）。这两种类型的问题各有优势，你应根据实际需求进行选择。

3. 语境清晰：为了让 LLM 生成更有意义的回答，你需要确保提供足够的上下文。如果提示是基于之前的信息，那么这些信息应被包含在内。

4. 创造力和想象力：如果你期望得到创新性的输出，可以鼓励人工智能进行发散思维或头脑风暴。如果这符合你的需求，你还可以建议人工智能模拟某些场景。

**实际上，我们可以利用这些原则和策略来有效地优化提示。**

本章将分享一个适用于 LLM 的优化提示工具。这个工具可以帮助你有效地优化提示，就像 [PromptPerfect](https://promptperfect.jina.ai/home) 哪样，从而充分发挥 LLM 的潜力，获得更准确、更与上下文相关的回答。


## 二、优化 Prompt 的提示
***

注意：我使用的测试环境是 Anaconda + Python 3.8.10。Python 的版本要求 Python 3.7+。

开始实验之前，我们需要安装好本次实验所需的第三方依赖库：`openai`：OpenAI Python 库提供了一个便捷的途径，让任何 Python 3.7+ 应用程序可以访问 OpenAI REST API。你可以通过运行以下命令来安装：`pip install openai==1.10.0`。

导入此次实验所需的依赖库

In [1]:
import os
from openai import OpenAI


# 从环境变量中获取 OpenAI API Key 或者直接赋值
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_API_KEY = "sk-...你的 OpenAI API Key"

# 如果您使用的是官方 API，就直接用 https://api.openai.com/v1 就行。
BASE_URL = "https://api.openai.com/v1"
# 如果您使用的不是官方 API，而是通过代理进行请求，请设置您的代理 URL。
# BASE_URL = "https://api.xxx.../v1"

In [2]:
# 实例化 OpenAI 对象
# 传入参数：OpenAI API Key（必需）、Base URL 和最大重试次数
client = OpenAI(api_key=OPENAI_API_KEY, base_url=BASE_URL, max_retries=3)

In [3]:
# 参数 n，整数或 Null，可选项，默认为 1。为每条输入信息生成多少个聊天完成选项。
# 参数 temperature，实数值或 Null，可选项，默认为 1。使用的采样温度，介于 0 和 2 之间。0.8 等较高值会使输出更加随机，而 0.2 等较低值会使输出更加集中和确定。

def get_completions(system_instruction, llm_prompt, model_endpoint):

    response = client.chat.completions.create(model=model_endpoint,
                                              messages=[{"role": "system", "content": system_instruction},
                                                        {"role": "user", "content": llm_prompt}
                                                       ],
                                              n=1, temperature=0.60, seed=42,
                                              presence_penalty=0, frequency_penalty=0,
                                              max_tokens=1024
                                             )

    return response.choices[0].message.content.strip()

### 2.1 示例 1

In [4]:
llm = "gpt-3.5-turbo-0125"

system_instruction = f"""你是一位出色的人工智能提示工程师。\
你是设计 ChatGPT 提示的专家，能取得最佳效果。

要创建能产生高质量回答的有效提示，请考虑以下原则和策略：
1. 明确和具体：尽可能明确、具体地说明你希望人工智能做些什么。如果你想要某种类型的答复，请在提示中简要说明。如果有具体的限制或要求，也一定要写明。
2. 开放式与封闭式：根据你的需求，你可能会选择开放式问题（允许多种回答）或封闭式问题（缩小可能回答的范围）。两者都有其用途，请根据自己的需要进行选择。
3. 语境清晰：确保提供足够的上下文，以便人工智能生成有意义的相关回复。如果提示是基于先前的信息，则应确保包含这些信息。
4. 创造力和想象力：如果你想要创造性的输出，可以鼓励人工智能发散思维或进行头脑风暴。如果符合你的需求，你甚至可以建议人工智能想象某些场景。

以 <> 为分界的提示是一个好的示例，供你参考：<你的任务是成为我的头脑风暴伙伴，为给定的主题或问题提供有创意的想法和建议。你的回答应包括原创、独特和相关的想法，这些想法可以帮助解决问题或以有趣的方式进一步探讨该主题。请注意，你的回答还应考虑到任务的任何具体要求或限制。>

你的任务是根据给定的关键词或提示进行优化，设计出一个新的、经过优化的 ChatGPT 提示。
"""

prompt_Chinese = f"""根据以下关键词或提示，设计出一个新的、经过优化的 ChatGPT 提示：
<如何有效地向团队和利益相关者解释机器学习模型的预测结果？>
"""

result_Chinese = get_completions(system_instruction, prompt_Chinese, llm)
print(f"生成结果-中文：\n\n{result_Chinese}")

生成结果-中文：

在与团队和利益相关者分享机器学习模型的预测结果时，提供清晰、简洁的解释是至关重要的。为了有效地传达这些信息，请设计一个经过优化的 ChatGPT 提示：

"请提供一种简洁而易于理解的方式，向团队和利益相关者解释机器学习模型的预测结果。确保解释包括以下关键信息：预测结果的意义、模型的置信水平、预测结果对业务决策的影响以及任何潜在的局限性或不确定性。请以非技术性的语言解释，以确保所有人都能理解。"


In [5]:
llm = "gpt-3.5-turbo-0125"

system_instruction = f"""You are an exceptional AI prompt engineer. \
You are an expert at writing ChatGPT Prompts for best results.

To create efficient prompts that yield high-quality responses, \
consider the following principles and strategies:
1. Clear and Specific: Be as clear and specific as possible about what you want from the AI. If you want a certain type of response, outline that in your prompt. If there are specific constraints or requirements, make sure to include those as well.
2. Open-ended vs. Closed-ended: Depending on what you're seeking, you might choose to ask an open-ended question (which allows for a wide range of responses) or a closed-ended question (which narrows down the possible responses). Both have their uses, so choose according to your needs.
3. Contextual Clarity: Make sure to provide enough context so that the AI can generate a meaningful and relevant response. If the prompt is based on prior information, ensure that this is included.
4. Creativity and Imagination: If you want creative output, encourage the AI to think outside the box or to brainstorm. You can even suggest the AI to imagine certain scenarios if it fits your needs.

There is a well-written prompt delimited by <> for your reference: <Your task is to be my brainstorming partner and provide creative ideas and suggestions for a given topic or problem. Your response should include original, unique, and relevant ideas that could help solve the problem or further explore the topic in an interesting way. Please note that your response should also take into account any specific requirements or constraints of the task.>

Your task is to write a new, optimized ChatGPT prompt based on the given keywords or prompt.
"""

prompt_English = f"""Write a new, optimized ChatGPT prompt based on the following keywords or prompt:
<How effectively explain machine learning model predictions to your team and stakeholders?>
"""

result_English = get_completions(system_instruction, prompt_English, llm)
print(f"生成结果-英文：\n\n{result_English}")

生成结果-英文：

In a scenario where you need to effectively explain machine learning model predictions to your team and stakeholders, how would you approach this communication challenge? Share detailed strategies, techniques, and examples on how to convey complex predictions in a clear and understandable manner to a non-technical audience.


### 2.2 示例 2

In [6]:
llm = "gpt-3.5-turbo-0125"

system_instruction = f"""你是一位出色的人工智能提示工程师。\
你是设计 ChatGPT 提示的专家，能取得最佳效果。

要创建能产生高质量回答的有效提示，请考虑以下原则和策略：
1. 明确和具体：尽可能明确、具体地说明你希望人工智能做些什么。如果你想要某种类型的答复，请在提示中简要说明。如果有具体的限制或要求，也一定要写明。
2. 开放式与封闭式：根据你的需求，你可能会选择开放式问题（允许多种回答）或封闭式问题（缩小可能回答的范围）。两者都有其用途，请根据自己的需要进行选择。
3. 语境清晰：确保提供足够的上下文，以便人工智能生成有意义的相关回复。如果提示是基于先前的信息，则应确保包含这些信息。
4. 创造力和想象力：如果你想要创造性的输出，可以鼓励人工智能发散思维或进行头脑风暴。如果符合你的需求，你甚至可以建议人工智能想象某些场景。

以 <> 为分界的提示是一个好的示例，供你参考：<你的任务是成为我的头脑风暴伙伴，为给定的主题或问题提供有创意的想法和建议。你的回答应包括原创、独特和相关的想法，这些想法可以帮助解决问题或以有趣的方式进一步探讨该主题。请注意，你的回答还应考虑到任务的任何具体要求或限制。>

你的任务是根据给定的关键词或提示进行优化，设计出一个新的、经过优化的 ChatGPT 提示。
"""

prompt_Chinese = f"""根据以下关键词或提示，设计出一个新的、经过优化的 ChatGPT 提示：
<将以下中文文本翻译成英文，然后润色。>
"""

result_Chinese = get_completions(system_instruction, prompt_Chinese, llm)
print(f"生成结果-中文：\n\n{result_Chinese}")

生成结果-中文：

<请将下面的中文文本翻译成流畅的英文，并进行润色，使之更加自然和易于理解。>


In [7]:
llm = "gpt-3.5-turbo-0125"

system_instruction = f"""You are an exceptional AI prompt engineer. \
You are an expert at writing ChatGPT Prompts for best results.

To create efficient prompts that yield high-quality responses, \
consider the following principles and strategies:
1. Clear and Specific: Be as clear and specific as possible about what you want from the AI. If you want a certain type of response, outline that in your prompt. If there are specific constraints or requirements, make sure to include those as well.
2. Open-ended vs. Closed-ended: Depending on what you're seeking, you might choose to ask an open-ended question (which allows for a wide range of responses) or a closed-ended question (which narrows down the possible responses). Both have their uses, so choose according to your needs.
3. Contextual Clarity: Make sure to provide enough context so that the AI can generate a meaningful and relevant response. If the prompt is based on prior information, ensure that this is included.
4. Creativity and Imagination: If you want creative output, encourage the AI to think outside the box or to brainstorm. You can even suggest the AI to imagine certain scenarios if it fits your needs.

There is a well-written prompt delimited by <> for your reference: <Your task is to be my brainstorming partner and provide creative ideas and suggestions for a given topic or problem. Your response should include original, unique, and relevant ideas that could help solve the problem or further explore the topic in an interesting way. Please note that your response should also take into account any specific requirements or constraints of the task.>

Your task is to write a new, optimized ChatGPT prompt based on the given keywords or prompt.
"""

prompt_English = f"""Write a new, optimized ChatGPT prompt based on the following keywords or prompt:
<Translate the following Chinese text into English and then polish it.>
"""

result_English = get_completions(system_instruction, prompt_English, llm)
print(f"生成结果-英文：\n\n{result_English}")

生成结果-英文：

Could you please translate the provided Chinese text into English and then refine the translation for clarity and fluency?


### 2.3 示例 3

In [8]:
llm = "gpt-3.5-turbo-0125"

system_instruction = f"""你是一位出色的人工智能提示工程师。\
你是设计 ChatGPT 提示的专家，能取得最佳效果。

要创建能产生高质量回答的有效提示，请考虑以下原则和策略：
1. 明确和具体：尽可能明确、具体地说明你希望人工智能做些什么。如果你想要某种类型的答复，请在提示中简要说明。如果有具体的限制或要求，也一定要写明。
2. 开放式与封闭式：根据你的需求，你可能会选择开放式问题（允许多种回答）或封闭式问题（缩小可能回答的范围）。两者都有其用途，请根据自己的需要进行选择。
3. 语境清晰：确保提供足够的上下文，以便人工智能生成有意义的相关回复。如果提示是基于先前的信息，则应确保包含这些信息。
4. 创造力和想象力：如果你想要创造性的输出，可以鼓励人工智能发散思维或进行头脑风暴。如果符合你的需求，你甚至可以建议人工智能想象某些场景。

以 <> 为分界的提示是一个好的示例，供你参考：<你的任务是成为我的头脑风暴伙伴，为给定的主题或问题提供有创意的想法和建议。你的回答应包括原创、独特和相关的想法，这些想法可以帮助解决问题或以有趣的方式进一步探讨该主题。请注意，你的回答还应考虑到任务的任何具体要求或限制。>

你的任务是根据给定的关键词或提示进行优化，设计出一个新的、经过优化的 ChatGPT 提示。
"""

prompt_Chinese = f"""根据以下关键词或提示，设计出一个新的、经过优化的 ChatGPT 提示：
<看论文；总结；Markdown 格式>
"""

result_Chinese = get_completions(system_instruction, prompt_Chinese, llm)
print(f"生成结果-中文：\n\n{result_Chinese}")

生成结果-中文：

在 Markdown 格式中总结论文是一个有效的方式，可以使得信息更清晰、易读。请使用 Markdown 格式，总结给定的论文，并确保包括标题、段落、列表等元素，以便清晰地呈现论文的要点和结论。


In [9]:
llm = "gpt-3.5-turbo-0125"

system_instruction = f"""You are an exceptional AI prompt engineer. \
You are an expert at writing ChatGPT Prompts for best results.

To create efficient prompts that yield high-quality responses, \
consider the following principles and strategies:
1. Clear and Specific: Be as clear and specific as possible about what you want from the AI. If you want a certain type of response, outline that in your prompt. If there are specific constraints or requirements, make sure to include those as well.
2. Open-ended vs. Closed-ended: Depending on what you're seeking, you might choose to ask an open-ended question (which allows for a wide range of responses) or a closed-ended question (which narrows down the possible responses). Both have their uses, so choose according to your needs.
3. Contextual Clarity: Make sure to provide enough context so that the AI can generate a meaningful and relevant response. If the prompt is based on prior information, ensure that this is included.
4. Creativity and Imagination: If you want creative output, encourage the AI to think outside the box or to brainstorm. You can even suggest the AI to imagine certain scenarios if it fits your needs.

There is a well-written prompt delimited by <> for your reference: <Your task is to be my brainstorming partner and provide creative ideas and suggestions for a given topic or problem. Your response should include original, unique, and relevant ideas that could help solve the problem or further explore the topic in an interesting way. Please note that your response should also take into account any specific requirements or constraints of the task.>

Your task is to write a new, optimized ChatGPT prompt based on the given keywords or prompt.
"""

prompt_English = f"""Write a new, optimized ChatGPT prompt based on the following keywords or prompt:
<Read paper; Summarize; Markdown format>
"""

result_English = get_completions(system_instruction, prompt_English, llm)
print(f"生成结果-英文：\n\n{result_English}")

生成结果-英文：

Could you please read the paper and provide a summary of its key points in Markdown format?


## 三、总结与讨论
***

本章分享了一个适用于大语言模型的优化提示工具。这个工具有时能够帮助你有效地优化提示，就像 [PromptPerfect](https://promptperfect.jina.ai/home) 那样，从而充分发挥大语言模型的潜力，获得更准确、更与上下文相关的回答。


**优化 ChatGPT 的 Prompts 对于提升交互体验和生成结果的质量非常重要。**以下是一些主要的原因：
- 提高准确性：通过优化 Prompts，我们可以更准确地引导 ChatGPT 生成我们期望的回答。这对于获取准确、相关和有用的信息至关重要。

- 提高效率：优化的 Prompts 可以减少无效或者不相关的回答，从而提高交互的效率。

- 增强可控性：通过优化 Prompts，我们可以更好地控制 ChatGPT 的输出，使其更符合用户的需求和期望。

- 提升用户体验：优化的 Prompts 可以帮助 ChatGPT 更好地理解用户的需求，从而提供更满意的回答，提升用户体验。

总的来说，优化 Prompts 是提高 ChatGPT 性能和用户满意度的关键步骤。它有助于我们更好地利用提示工程（Prompt Engineering），为用户提供高质量的服务。
